# Curso de Estadística Computacional con Python

![Portada](images/Portada.jpeg)

# Tabla de Contenido
- [Programación dinámica](#programación-dinámica)
    - [Optimzación Fibonnaci](#optimización-de-fibonacci)
        - [Optimización Especial](#optimización-especial)
        - [Código](#código)
    - [Caminos Aleatorios](#caminos-aleatorios)
        - [Resultados](#resultado-de-la-simulacion)
        - [Código](#códigos-del-borracho)
        - [Aporte solo pasos del borracho](#código-para-solo-observar-los-pasos-del-borracho)
- [Procesos Estocásticos](#procesos-estocásticos)
- [Monte Carlos](#monte-carlos)
- [Muestreo e Intervalos de Confianza](#muestro-e-intervalos-de-confianza)
- [Datos experimentales](#datos-experimentales)
- [Conclusiones](#conclusiones)


# Programación dinámica

**Programación dinámica:** Se escogió para esconder a patrocinadores gubernamentales el hecho que en realidad estaba haciendo Matemáticas. La frase Programación Dinámica es algo que ningún congresista puede oponerse. **Richard Bellman**

**Programación dinámica:**
- Subsestructura Óptima: Una solución global óptima se puede encontrar al combinar soluciones óptimas de subproblemas locales.
- Problemas empalmados: Una solución óptima que involucra resolver el mismo problema en varias ocasiones.

**Memoization:**
- La memorización es una técnica para guardar cómputos previos y evitar realizarlos nuevamente.
- Normalmente se utiliza un diccionario, donde las consultas se pueden hacer en O(1).
- Intercambia tiempo por espacio.

> Ventaja de consulta de los Sets y Dict de O(1).

## Optimización de Fibonacci

$$F_n = F_{n-1} + F_{n-2} \quad \forall~n > 2$$

![Fibonacci](images/Fibonnaci.png)

Pero el algoritmo recursivo es ineficiente, debido a que crece de manera exponencial el tiempo de cómputo.

> **Hint:** Para medir tiempo de ejecución desde consola (Mi sistema operativo es Windows usando Git Bash):
>```shell
>time python src/programacion_dinamica-1.py
>```
Aprovechando el tiempo de consulta de los diccionarios optimizamos el tiempo de ejecución para calcular un número $n$ de Fibonacci.

>**Hint:** Para aumentar el tamaño de recursión y no sea un límite, añadimos la siguiente línea de código:
>```python
>sys.setrecursionlimit(10002)
>```

> Sitio Web para observar como funciona ambos algoritmos: [Dynamic Programming (Fibonacci)](https://www.cs.usfca.edu/~galles/visualization/DPFib.html)


### Optimización Especial

In [2]:
from functools import lru_cache

@lru_cache(maxsize=None)
def fib(n):
    if n < 2:
        return n
    return fib(n-1) + fib(n-2)

### Código 
[**File**: programacion_dinamica-1.py](src/programacion_dinamica-1.py)

In [3]:
import sys
from functools import lru_cache

def fibonacci_recursivo(n):
    if n == 0 or n == 1:
        return 1

    return fibonacci_recursivo(n - 1) + fibonacci_recursivo(n - 2)


def fibonacci_dinamico(n, memo = {}):
    if n == 0 or n == 1:
        return 1

    try:
        return memo[n]
    except KeyError:
        # Si no esta en el diccionario, lo calculamos y lo guardamos
        resultado = fibonacci_dinamico(n - 1, memo) + fibonacci_dinamico(n - 2, memo)
        memo[n] = resultado

        return resultado

@lru_cache(maxsize=None)
def fib(n):
    if n < 2:
        return n
    return fib(n-1) + fib(n-2)

sys.setrecursionlimit(10002)
n = int(input('Escoge un numero: '))
# resultado = fibonacci_recursivo(n)
resultado = fib(n)
# resultado = fibonacci_dinamico(n)
print(f"Number: {n} -> Result: {resultado}")

Number: 1 -> Result: 1


## Caminos Aleatorios

> Caminos Aleatorios:
> - Es un tipo de simulación que elige aleatoriamente una decisión dentro de un conjunto de decisiones válidas.
> - Se utiliza en muchos campos del conocimiento cuando los sistemas no son deterministas e incluyen elementos de aleatoriedad.

La idea viene dado por el movimiento browniano, este es el movimiento aleatorio de partículas suspendidas en un fluido (líquido o gas) debido a colisiones con moléculas del medio. Este fenómeno fue observado por Robert Brown en 1827 y es un ejemplo de un proceso estocástico

### Entendiendo la aleatoriedad con Python

Se dividio en 3 clases:
- ¿Qué es un borracho? *Borracho*
- ¿Dónde se mueve este borracho? *Campo*
- Concepto abstracto a tráves de un objeto: *Coordenada*.

**Borracho**, toma cambios en alguna de las coordenadas de forma aleatoria. Cada posible paso tiene igual probabilidad de ocurrencia.

In [4]:
import random

class Borracho:

    def __init__(self, nombre):
        self.nombre = nombre


class BorrachoTradicional(Borracho):

    def __init__(self, nombre):
        super().__init__(nombre)

    def camina(self):
        # Igual probabilidad seleccionar cualquier paso
        return random.choice([(0, 1), (0, -1), (1, 0), (-1, 0)])

**Campo**, muestra (gráfica) los pasos del borracho y limita el área donde se mueve.

In [5]:
class Campo:

    def __init__(self):
        self.coordenadas_de_borrachos = {}

    def anadir_borracho(self, borracho, coordenada):
        """
        Add Borracho al campo
        """
        self.coordenadas_de_borrachos[borracho] = coordenada

    def mover_borracho(self, borracho):
        """
        Mueve el borracho modificando sus coordenadas
        """
        # Genera un paso
        delta_x, delta_y = borracho.camina()
        # Obtiene las coordenadas del borracho
        coordenada_actual = self.coordenadas_de_borrachos[borracho]
        # Crea una nueva coordena
        nueva_coordenada = coordenada_actual.mover(delta_x, delta_y)
        # Modifica la coordena
        self.coordenadas_de_borrachos[borracho] = nueva_coordenada

    def obtener_coordenada(self, borracho):
        """
        Get Coordenada
        """
        return self.coordenadas_de_borrachos[borracho]

**Coordenada**, crea el marco de referencia para el *Borracho* y el *Campo*. Nos permite también calcular la distancia entre una coordena y otra.

In [6]:
class Coordenada:

    def __init__(self, x, y):
        self.x = x
        self.y = y

    def mover(self, delta_x, delta_y):
        """
        Retorna una nueva Coordenada
        """
        return Coordenada(self.x + delta_x, self.y + delta_y)

    def distancia(self, otra_coordenada):
        """
        Calcula la distancia Euclidiana
        """
        delta_x = self.x - otra_coordenada.x 
        delta_y = self.y - otra_coordenada.y 

        return (delta_x**2 + delta_y**2)**0.5

> La simulación genera $n$ cantidad de pasos del borracho. Siempre que hacemos aleatoriedad ejecutamos $m$ simulaciones para $n$ pasos. Para luego obtener las estadísticas necesarias para proponer los resultados de esas observaciones.

In [7]:
# Librerias a emplear
from src.borracho import BorrachoTradicional
from src.campo import Campo 
from src.coordenada import Coordenada 

# Para graficar
from bokeh.plotting import figure, show

In [8]:
def caminata(campo, borracho, pasos):
    # Coordinate init
    inicio = campo.obtener_coordenada(borracho)

    for _ in range(pasos):
        # Move borracho
        campo.mover_borracho(borracho)

    # Return distance between origin and end
    return inicio.distancia(campo.obtener_coordenada(borracho))

In [9]:
def simular_caminata(pasos, numero_de_intentos, tipo_de_borracho):
    
    # Add Borracho
    borracho = tipo_de_borracho(nombre='Rigo')
    # Crear origen
    origen = Coordenada(0, 0)
    # Save distance
    distancias = []

    for _ in range(numero_de_intentos):
        # Creo el campo
        campo = Campo()
        # Ubico el borracho
        campo.anadir_borracho(borracho, origen)
        # Caminar y devuelve la distancia caminda desde el origen
        simulacion_caminata = caminata(campo, borracho, pasos)
        # Save
        distancias.append(round(simulacion_caminata, 1))

    return distancias

In [10]:
def graficar(x, y):
    """
    Plot n vs Distancia media
    """
    grafica = figure(title='Camino aleatorio', x_axis_label='pasos', 
                     y_axis_label='distancia')
    grafica.line(x, y, legend_label='distancia media')

    show(grafica)

In [11]:
def main(distancias_de_caminata, numero_de_intentos, tipo_de_borracho):

    # Save mean distance
    distancias_media_por_caminata = []

    for pasos in distancias_de_caminata:
        distancias = simular_caminata(pasos, numero_de_intentos, tipo_de_borracho)
        # Media
        distancia_media = round(sum(distancias) / len(distancias), 4)
        # Maxima
        distancia_maxima = max(distancias)
        # Minima
        distancia_minima = min(distancias)
        # Save
        distancias_media_por_caminata.append(distancia_media)
        print(f'{tipo_de_borracho.__name__} caminata aleatoria de {pasos} pasos')
        print(f'Media = {distancia_media}')
        print(f'Max = {distancia_maxima}')
        print(f'Min = {distancia_minima}')

    graficar(distancias_de_caminata, distancias_media_por_caminata)

In [12]:
# Number of steps -> n
distancias_de_caminata = [10, 100, 1000, 10000]
# Number of Simulations -> m
numero_de_intentos = 100

main(distancias_de_caminata, numero_de_intentos, BorrachoTradicional)

BorrachoTradicional caminata aleatoria de 10 pasos
Media = 2.887
Max = 7.1
Min = 0.0
BorrachoTradicional caminata aleatoria de 100 pasos
Media = 8.723
Max = 21.3
Min = 0.0
BorrachoTradicional caminata aleatoria de 1000 pasos
Media = 29.163
Max = 70.7
Min = 6.3
BorrachoTradicional caminata aleatoria de 10000 pasos
Media = 90.969
Max = 226.9
Min = 15.0


### Resultado de la Simulacion

![Distancias Medias](images/camino_aleatorio.png)

### Códigos del Borracho

|Name File | Link |
|-|-|
|Borracho| [Borracho](src/borracho.py) |
|Campo| [Campo](src/campo.py) |
|Coordenadas | [Coordenadas](src/coordenada.py)|
|Simulación | [Simulation](src/camino_aleatorio.py)|

### Código para solo observar los pasos del borracho

> Nota: Aportado por un estudiante de la comunidad de Platzi. (Angel Armando Martínez Blanco)

In [13]:
from src.borracho import BorrachoTradicional
from src.coordenada import Coordenada
from src.campo import Campo

from bokeh.plotting import figure, show

In [14]:
def graficar(x, y):
    figura = figure()
    figura.line(x, y)
    show(figura)

In [15]:
def ejecutar_caminata(campo, borracho, distancia):
    x_arreglo = []
    y_arreglo = []
    x_arreglo.append(campo.obtener_coordenada(borracho).x)
    y_arreglo.append(campo.obtener_coordenada(borracho).y)
    for _ in range(distancia):
        campo.mover_borracho(borracho) #se actualiza las coordenadas del borracho
        x_arreglo.append(campo.obtener_coordenada(borracho).x)
        y_arreglo.append(campo.obtener_coordenada(borracho).y)
    graficar(x_arreglo, y_arreglo)

In [16]:
def main(distancia, inicio, borracho):
    campo = Campo()
    campo.anadir_borracho(borracho, inicio) #poner un borracho en origen
    ejecutar_caminata(campo, borracho, distancia)

In [17]:
distancia = 100000
inicio = Coordenada(0,0)
borracho = BorrachoTradicional('Rigo')
main(distancia, inicio, borracho)

![Resultado Caminta Borracho](images/borracho_caminata.png)

# Programas Estocásticos

- Un programa es determinístico si cuando se corre con el mismo input produce el mismo output.
- Los programas determinísticos son muy importantes, pero existen problemas que no pueden resolverse de esa manera.
- La programación estocástica permite introducir aleatoriedad a nuestros programas para crear simulaciones que permiten resolver otro tipo de problemas.

> Los programas estocásticos se aprovechan de que las distribuciones probabilísticas de un problema se conocen o pueden ser estimadas.

## Probabilidades

- La probabilidad es una medida de la certidumbre asociada a un evento o suceso futuro y suele expresarse como un número entre 0 y 1.
- Una probabilidad de 0 significa que un suceso jamás sucederá.
- Una probabilidad de 1 significa que un suceso está garantizado de suceder en el futuro.
- Al hablar de probabilidad preguntamos qué fracción de todos los posibles eventos tiene la propiedad que buscamos.
- Por eso es importante poder calcular todas las posibilidades de un evento para entender su probabilidad.
- La probabilidad de que un evento suceda y de que no suceda es siempre 1.

Ley del Complemento:
$$P(A) + P(~A) = 1$$ 

Ley multiplicativa:
$$P(A \text{y} B) = P(A) * P(B)$$



Ley aditiva 
Mutuamente exclusivos:
$$P(A \text{o} B) = P(A) + P(B)$$
No exclusiva:
$$P(A \text{o} B) = P(A) + P(B) - P(A \text{y} B)$$

In [18]:
import random

In [19]:
def tirar_dado(numero_de_tiros):

    secuencia_de_tiros = []

    for _ in range(numero_de_tiros):
        tiro = random.choice([1, 2, 3, 4, 5, 6])
        secuencia_de_tiros.append(tiro)

    return secuencia_de_tiros

In [26]:
def main(numero_de_tiros, numero_de_intentos):
    
    tiros = []
    for _ in range(numero_de_intentos):
        secuencia_de_tiros = tirar_dado(numero_de_tiros)
        tiros.append(secuencia_de_tiros)
    

    tiros_con_1 = 0
    for tiro in tiros:
        # Para obtener el complemento
        # if 1 not in tiro:
        if 1 in tiro:
            tiros_con_1 += 1

    probabilidad_tiros_con_1 = tiros_con_1 / numero_de_intentos
    print(f"Probabilidad de obtener por lo" +
          f" menos un 1 en {numero_de_tiros}" +
          f" tiros = {probabilidad_tiros_con_1}")

In [27]:
numero_de_tiros = int(input('Cuantas tiros del dado: '))
numero_de_intentos = int(input('Cuantas veces correra la simulacion: '))

main(numero_de_tiros, numero_de_intentos)

Probabilidad de obtener por lo menos un 1 en 1 tiros = 0.147


In [22]:
# Probabilidad de no obtener un 1
(5/6)**10

0.1615055828898458

In [23]:
# Probabilidad de obtener al menos un 1
1 - (5/6)**10

0.8384944171101543

In [24]:
## Ejecutar código directamente
%run src/probabilidades.py

Probabilidad de no obtener por lo menos un 1 en 1 tiros = 0.824


#### Reto 
> Obtener la probabilidad de obtener un 12 al tirar 2 dados

La posicion más justa es tener todas las posibles combinaciones y de ahi elegir la tirada. Eligiendo una única vez por dados para luego sumar no es muy realista, serían como momentos separados.

In [28]:
def reto(numero_de_tiros, numero_de_intentos):
    
    all_valores = [2,3,4,5,6,7,3,4,5,6,7,8,4,
                   5,6,7,8,9,5,6,7,8,9,10,6,7,
                   8,9,10,11,7,8,9,10,11,12]
    
    tiros = []
    for _ in range(numero_de_intentos):
        secuencia_de_tiros = []
        for _ in range(numero_de_tiros):
            tiro = random.choice(all_valores)
            secuencia_de_tiros.append(tiro)

        tiros.append(secuencia_de_tiros)

    tiros_con_12 = 0
    for tiro in tiros:
        if 12 in tiro:
            tiros_con_12 += 1
    
   
    probabilidad_tiros_con_1 = tiros_con_12 / numero_de_intentos
    print(f"Probabilidad de obtener por lo" +
          f" menos un 12 en {numero_de_tiros}" +
          f" tiros = {probabilidad_tiros_con_1}")

In [31]:
numero_de_tiros = int(input('Cuantas tiros del dado: '))
numero_de_intentos = int(input('Cuantas veces correra la simulacion: '))

reto(numero_de_tiros, numero_de_intentos)

Probabilidad de obtener por lo menos un 12 en 10 tiros = 0.24735


## Inferencia Estadística

- Con las simulaciones podemos calcular las probabilidades de eventos complejos sabiendo las probabilidades de eventos simples.
- ¿Qué pasa cuando no sabemos las probabilidades de los eventos simples?
- Las técnicas de la Inferencia Estadística nos permiten inferir/concluir las propiedades de una población a partir de una muestra aleatoria.

> El principio guía de la Inferencia Estadística es que una muestra aleatoria tiende a exhibir las mismas propiedades que la población de la cual fue extraída. **John Guttag**

# Monte Carlos

# Muestro e Intervalos de Confianza

# Datos Experimentales

# Conclusiones
